In [1]:
import torch
import numpy
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import os

from model.net import *
from utils.training import *
from data.data import *

## Experiment Config

In [2]:
model_path = os.getcwd()

args = {
    'USE_CUDA': True if torch.cuda.is_available() else False,
    'BATCH_SIZE': 256,
    'N_EPOCHS': 30,
    'LEARNING_RATE': 1e-2,
    'MOMENTUM': 0.9,
    'DATASET_NAME':'mnist',
    'LAMBDA_recon': 0.5, #0.0005
    'LAMBDA_margin': 0,
    'LReLU_negative_slope':0.1,
    'WEIGHTDECAY':0,#5e-3,
}

## Model Loading

In [3]:
#Config for 49 16d vectors in the Primary Capsule. Set Softmax dimension to 0 in this case
class Config:
    def __init__(self):
        # CNN (cnn)
        self.cnn_in_channels = 1
        self.cnn_out_channels = 12
        self.cnn_kernel_size = 15

        # Primary Capsule (pc)
        self.pc_num_capsules = 16
        self.pc_in_channels = 12
        self.pc_out_channels = 1
        self.pc_kernel_size = 8
        self.pc_num_routes = 1 * 7 * 7

        # Digit Capsule 1 (dc)
        self.dc_num_capsules = 49
        self.dc_num_routes = 1 * 7 * 7
        self.dc_in_channels = 16
        self.dc_out_channels = 16
        
        # Digit Capsule 2 (dc)
        self.dc_2_num_capsules = 10
        self.dc_2_num_routes = 1 * 7 * 7
        self.dc_2_in_channels = 16
        self.dc_2_out_channels = 16

        # Decoder
        self.input_width = 28
        self.input_height = 28

torch.manual_seed(1)
config = Config()

net = CapsNet(args, config)
# capsule_net = torch.nn.DataParallel(capsule_net)
if args['USE_CUDA']:
    net = net.cuda()
    
# # freeze All layers except Decoder
to_freeze = [net.conv_layer, net.primary_capsules, net.digit_capsules_1, net.digit_capsules_2]
for layer in to_freeze:
    for param in layer.parameters():
        param.requires_grad = False
    layer.eval()
    
net.load_state_dict(torch.load(os.path.join(model_path, 'CapsNetMNIST.pth'), map_location='cpu'))

<All keys matched successfully>

## Loading Dataset

In [4]:
trainloader, testloader = dataset(args)

## Training CapsuleNet

In [5]:
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()), lr=args['LEARNING_RATE'], weight_decay=args['WEIGHTDECAY'])

def train_CapsNet(model, optimizer, train_loader, epoch, args):
    capsule_net = model
    capsule_net.train()
    n_batch = len(train_loader)
    total_loss = 0
    for batch_id, (data, target) in enumerate(tqdm(train_loader)):
        target = torch.sparse.torch.eye(10).index_select(dim=0, index=target)
        if(args['USE_CUDA']):
            data, target = data.cuda(), target.cuda()

        optimizer.zero_grad()
        output, reconstructions, masked = capsule_net(data)
        loss = capsule_net.loss(data, output, target, reconstructions)
        if(batch_id%100==0):
            print(capsule_net.decoder.reconstraction_layers[0].weight[0][:3])
        loss.backward()
        if(batch_id%100==0):
            print("the grads before are", capsule_net.decoder.reconstraction_layers[0].weight.grad[0][:3].data)
        optimizer.step()
        if(batch_id%100==0):
            print(capsule_net.decoder.reconstraction_layers[0].weight[0][:3])
            print("the grads after are", capsule_net.decoder.reconstraction_layers[0].weight.grad[0][:3].data)
        correct = torch.sum(torch.argmax(masked, 1) == torch.argmax(target, 1))
        train_loss = loss.item()
        total_loss += train_loss
        if batch_id % 100 == 0:
            tqdm.write("Epoch: [{}/{}], Batch: [{}/{}], train accuracy: {:.6f}, loss: {:.6f}".format(
                epoch,
                args['N_EPOCHS'],
                batch_id + 1,
                n_batch,
                correct / float(args['BATCH_SIZE']),
                train_loss / float(args['BATCH_SIZE'])
                ))
    tqdm.write('Epoch: [{}/{}], train loss: {:.6f}'.format(epoch,args['N_EPOCHS'],total_loss / len(train_loader.dataset)))

for e in range(1, args['N_EPOCHS'] + 1):
    train_CapsNet(net, optimizer, trainloader, e, args)
    test_CapsNet(net, testloader, e, args)

  1%|▏         | 3/235 [00:00<02:05,  1.84it/s]

tensor([-0.1905, -0.1794, -0.0265], device='cuda:0', grad_fn=<SliceBackward>)
the grads before are tensor([-0.0002,  0.0003,  0.0002], device='cuda:0')
tensor([-0.1805, -0.1894, -0.0365], device='cuda:0', grad_fn=<SliceBackward>)
the grads after are tensor([-0.0002,  0.0003,  0.0002], device='cuda:0')
Epoch: [1/30], Batch: [1/235], train accuracy: 0.992188, loss: 0.001790


 44%|████▍     | 103/235 [00:08<00:10, 13.19it/s]

tensor([-0.1836, -0.1958, -0.0241], device='cuda:0', grad_fn=<SliceBackward>)
the grads before are tensor([ 0.0007, -0.0009, -0.0008], device='cuda:0')
tensor([-0.1828, -0.1968, -0.0249], device='cuda:0', grad_fn=<SliceBackward>)
the grads after are tensor([ 0.0007, -0.0009, -0.0008], device='cuda:0')
Epoch: [1/30], Batch: [101/235], train accuracy: 0.976562, loss: 0.000983


 86%|████████▋ | 203/235 [00:15<00:02, 13.15it/s]

...

 86%|████████▋ | 203/235 [00:17<00:02, 11.37it/s]

tensor([-0.3354, -0.1809,  0.1146], device='cuda:0', grad_fn=<SliceBackward>)
the grads before are tensor([-1.3398e-05,  1.8941e-05,  1.3899e-05], device='cuda:0')
tensor([-0.3345, -0.1818,  0.1137], device='cuda:0', grad_fn=<SliceBackward>)
the grads after are tensor([-1.3398e-05,  1.8941e-05,  1.3899e-05], device='cuda:0')
Epoch: [30/30], Batch: [201/235], train accuracy: 0.968750, loss: 0.000812


100%|██████████| 235/235 [00:20<00:00, 11.41it/s]

Epoch: [30/30], train loss: 0.000791


Epoch: [30/30], test accuracy: 0.985100, loss: 0.207699


In [6]:
torch.save(net.state_dict(), "./CapsNetMNIST_Recon.pth")

In [7]:
for param in net.parameters():
    if(param.requires_grad==True):
        print(param.shape)

torch.Size([512, 160])
torch.Size([512])
torch.Size([1024, 512])
torch.Size([1024])
torch.Size([784, 1024])
torch.Size([784])


In [ ]:
for p in net.parameters():
    

In [10]:
torch.tensor(0.1307).div(2)

tensor(0.0654)

In [ ]:
from torchvision import transforms

a = transforms.Normalize((0.1307,), (0.3081,))
a.std

In [ ]:
#Config for 16 1d vectors in Capsule Layer. Set the Softmax Dimension to 1 in this case
# class Config:
#     def __init__(self, dataset='mnist'):
#         # CNN (cnn)
#         self.cnn_in_channels = 1
#         self.cnn_out_channels = 12
#         self.cnn_kernel_size = 15

#         # Primary Capsule (pc)
#         self.pc_num_capsules = 1
#         self.pc_in_channels = 12
#         self.pc_out_channels = 16
#         self.pc_kernel_size = 8
#         self.pc_num_routes = 16 * 7 * 7

#         # Digit Capsule 1 (dc)
#         self.dc_num_capsules = 49
#         self.dc_num_routes = 16 * 7 * 7
#         self.dc_in_channels = 1
#         self.dc_out_channels = 1 #16
        
#         # Digit Capsule 2 (dc)
#         self.dc_2_num_capsules = 10
#         self.dc_2_num_routes = 7 * 7
#         self.dc_2_in_channels = 1 #16
#         self.dc_2_out_channels = 16

#         # Decoder
#         self.input_width = 28
#         self.input_height = 28